# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# !pip install deepspeed

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-10faef8f-3ee1-c038-8ddb-4b704f4c3555)


### Setup the model

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

#modelname= "bigscience/bloom-560m",
#modelname= "bigscience/bloomz-3b",
#modelname= "bigscience/bloom-3b",
modelname= "bigscience/bloom-7b1"
model = AutoModelForCausalLM.from_pretrained(
    modelname,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


### Freezing the original weights


In [10]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [14]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

## Data

In [15]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

In [16]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [17]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

### Training

### Using DeepSpeed

In [ ]:
# from transformers import AdamW, get_linear_schedule_with_warmup

# import torch
# import deepspeed

# optimizer = AdamW(
#     model.parameters(),
#     eps=1e-8,  # Epsilon value for numerical stability (adjust if needed)
#     weight_decay=0.01  # Weight decay (adjust if needed)
# )

# # Load the DeepSpeed engine
# ds_engine, model, optimizer = deepspeed.initialize(model, optimizer, ds_config="/content/ds_config.json")


In [ ]:
# import deepspeed
# # optimizer = deepspeed.optimizers.AdamW(model.parameters(), lr=2e-4)

# trainer = deepspeed.Deeps(
#     model=model,
#     train_dataset=data['train'],
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=3,
#         gradient_accumulation_steps=2,
#         warmup_steps=100,
#         max_steps=200,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir='outputs'
#     ),
# )

### Using Pytorch

In [ ]:
# !export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:100

In [18]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.082700
2,3.194000
3,3.136600
4,3.376300
5,3.400300
6,3.500000
7,2.963800
8,3.154300
9,3.459200
10,3.121500


TrainOutput(global_step=200, training_loss=2.30858235180378, metrics={'train_runtime': 726.6021, 'train_samples_per_second': 4.404, 'train_steps_per_second': 0.275, 'total_flos': 1.3203052799655936e+16, 'train_loss': 2.30858235180378, 'epoch': 1.28})

error when r=16    
OutOfMemoryError: CUDA out of memory. Tried to allocate 3.40 GiB (GPU 0; 14.75 GiB total capacity; 11.38 GiB already allocated; 506.81 MiB free; 13.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


error when r=12    
OutOfMemoryError: CUDA out of memory. Tried to allocate 3.40 GiB (GPU 0; 14.75 GiB total capacity; 11.31 GiB already allocated; 1.02 GiB free; 12.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

error when r=12,  per_device_train_batch_size=2,  gradient_accumulation_steps=2,    
OutOfMemoryError: CUDA out of memory. Tried to allocate 1.70 GiB (GPU 0; 14.75 GiB total capacity; 12.05 GiB already allocated; 988.81 MiB free; 12.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

error when r=12, per_device_train_batch_size=2, gradient_accumulation_steps=2,
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:100
stopped at 43 step

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.70 GiB (GPU 0; 14.75 GiB total capacity; 12.05 GiB already allocated; 670.81 MiB free; 13.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Share adapters on the 🤗 Hub

In [19]:
model.push_to_hub("harithapliyal/bloom-7b1-lora-tagger",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harithapliyal/bloom-7b1-lora-tagger/commit/591716d736a56acf94ffe8bd2cc1a646f78ecc70', commit_message='basic training', commit_description='', oid='591716d736a56acf94ffe8bd2cc1a646f78ecc70', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [ ]:
# config.base_model_name_or_path=''
# #PeftConfig.from_pretrained(peft_model_id)
# config

In [ ]:
!pip install bitsandbytes

In [7]:
!pip install -q accelerate

In [ ]:
!pip install safetensors

In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "harithapliyal/bloom-7b1-lora-tagger"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True,
    load_in_8bit=False,
    device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


## Inference

In [8]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt').to('cuda')

# with torch.cuda.amp.autocast():
output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 “Training models with PEFT and LoRa is cool” ->:  “We are working on it”
“We are working on it” ->:  “We are working on it”
“We are working on it” ->:  “We are working on it”
“We are working on it


In [17]:

batch = tokenizer("Two things are infinite: the universe and human stupidity->: ", return_tensors='pt').to('cuda')

# with torch.cuda.amp.autocast():
output_tokens = model.generate(**batch, max_new_tokens=10)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Two things are infinite: the universe and human stupidity->:  The universe and human stupidity
I have a


In [16]:
batch = tokenizer("I'm selfish->: ", return_tensors='pt').to('cuda')

# with torch.cuda.amp.autocast():
output_tokens = model.generate(**batch, max_new_tokens=10)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 I'm selfish->:  I don't want to be a burden to my family


["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]